### ECE473: Elements of Decentralized Finance
### Kevin Jeon, Jeremy Kim, Jason Persaud, Nikolai Salazar

Final Project: Risk Parity Balancing for Multi-token ETF Arbitrage

In [ ]:
!pip install yfinance pandas numpy matplotlib statsmodels

### Cointegration Implementation

In [ ]:
import argparse
import logging
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller, coint

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# ---------------------------------------------------------------------
# 1. Data Download
# ---------------------------------------------------------------------
def download_data(tickers, start_date, end_date, interval="1d"):
    """
    Downloads price data from Yahoo Finance.
    Uses 'Adj Close' if available, otherwise uses 'Close'.
    """
    logging.info(f"Downloading data for {tickers} from {start_date} to {end_date}")
    df = yf.download(tickers, start=start_date, end=end_date, interval=interval, progress=False)
    if df.empty:
        logging.error(f"No data downloaded for tickers: {tickers}")
        raise RuntimeError(f"No data downloaded for {tickers}.")
    if "Adj Close" in df.columns:
        px = df["Adj Close"].copy()
    elif "Close" in df.columns:
        px = df["Close"].copy()
    else:
        px = df.copy()
    return px.ffill()  # forward-fill missing data

# ---------------------------------------------------------------------
# 2. Portfolio Construction (Daily Rebalancing)
#    - Market Cap Weighted
#    - Risk Parity (Inverse Volatility)
# ---------------------------------------------------------------------
def get_rebalance_times(prices, freq_str="1D"):
    """
    Generate rebalancing dates by aligning a generated date range to available trading days.
    """
    rb_dates = pd.date_range(start=prices.index[0], end=prices.index[-1], freq=freq_str)
    adjusted = []
    for t in rb_dates:
        idx = prices.index.searchsorted(t)
        if idx < len(prices.index):
            adjusted.append(prices.index[idx])
    return sorted(set(adjusted))

def simulate_rebalanced_portfolio(prices, weights_func, freq_str="1D", initial_value=1.0):
    """
    Simulates a portfolio that is rebalanced at specified dates using the provided weights_func.
    """
    rebalance_dates = get_rebalance_times(prices, freq_str)
    portfolio_values = pd.Series(index=prices.index, dtype=float)
    current_value = initial_value
    if prices.index[0] not in rebalance_dates:
        rebalance_dates = [prices.index[0]] + rebalance_dates
        rebalance_dates = sorted(set(rebalance_dates))
    logging.info(f"Performing {len(rebalance_dates)} rebalances.")
    for i in range(len(rebalance_dates)):
        rb_date = rebalance_dates[i]
        w = weights_func(prices, rb_date)
        if np.isnan(w).any():
            logging.warning(f"NaN weights encountered at {rb_date}, using equal weights.")
            w = np.ones(len(w)) / len(w)
        if i < len(rebalance_dates) - 1:
            next_rb_date = rebalance_dates[i+1]
        else:
            next_rb_date = prices.index[-1]
        sub_prices = prices.loc[rb_date:next_rb_date]
        base_px = prices.loc[rb_date]
        rel_returns = sub_prices.divide(base_px, axis=1)
        period_value = rel_returns.dot(w) * current_value
        portfolio_values.loc[sub_prices.index] = period_value
        current_value = period_value.iloc[-1]
    return portfolio_values

def constant_weights_func(prices, current_time, weights_dict):
    """
    Returns weights based on the provided dictionary.
    """
    return np.array([weights_dict.get(t, 0.0) for t in prices.columns])

def simulate_marketcap_portfolio(prices, weights_dict, freq_str="1D", initial_value=1.0):
    """
    Simulates a market cap weighted portfolio using provided constant weights.
    """
    def wfunc(px, date):
        return constant_weights_func(px, date, weights_dict)
    return simulate_rebalanced_portfolio(prices, wfunc, freq_str=freq_str, initial_value=initial_value)

def risk_parity_weights(prices, current_time, window=20, epsilon=1e-8):
    """
    Computes risk-parity weights using inverse volatility over a rolling window.
    """
    sub = prices.loc[:current_time].iloc[-window:]
    if sub.shape[0] < 2:
        n_assets = prices.shape[1]
        logging.debug(f"Insufficient data at {current_time}, using equal weights.")
        return np.ones(n_assets) / n_assets
    logret = np.log(sub / sub.shift(1)).dropna()
    vol = logret.std().fillna(epsilon)
    vol[vol == 0] = epsilon
    inv_vol = 1.0 / vol
    weights = inv_vol / inv_vol.sum()
    logging.debug(f"Risk-parity weights at {current_time}: {weights.to_dict()}")
    return weights.values

def simulate_risk_parity_portfolio(prices, window=20, freq_str="1D", initial_value=1.0):
    """
    Simulates a risk-parity portfolio using inverse-volatility weights.
    """
    return simulate_rebalanced_portfolio(
        prices,
        lambda px, date: risk_parity_weights(px, date, window=window),
        freq_str=freq_str,
        initial_value=initial_value
    )

# ---------------------------------------------------------------------
# 3. Tracking & Z-Score Metrics
# ---------------------------------------------------------------------
def compute_tracking_metrics(etf, synth, window=20):
    """
    Computes the tracking error and rolling correlation between ETF and synthetic portfolio.
    """
    etf, synth = etf.align(synth, join="inner")
    ret_e = etf.pct_change()
    ret_s = synth.pct_change()
    diff = ret_e - ret_s
    tracking_error = diff.rolling(window).std()
    rolling_corr = ret_e.rolling(window).corr(ret_s)
    return tracking_error, rolling_corr

def compute_spread_zscore(etf, synth, window=20):
    """
    Computes the z-score of the spread between ETF and synthetic portfolio.
    """
    etf, synth = etf.align(synth, join="inner")
    spread = etf - synth
    mu = spread.rolling(window).mean()
    sigma = spread.rolling(window).std()
    zscore = (spread - mu) / sigma
    return zscore

# ---------------------------------------------------------------------
# New: Cointegration Check
# ---------------------------------------------------------------------
def check_cointegration(series1, series2):
    """
    Performs a cointegration test between two time series using the Engle-Granger two-step method.
    Returns the test statistic and p-value.
    """
    # Align the two series to ensure they have the same index
    series1, series2 = series1.align(series2, join='inner')  # Use 'inner' join to keep only common dates

    score, pvalue, _ = coint(series1, series2)
    return score, pvalue

# ---------------------------------------------------------------------
# 4. Pairs Trading Implementation
# ---------------------------------------------------------------------
def pairs_backtest(etf, synth, z_entry=1.0, z_exit=0.0, window=20, transaction_cost=None):
    """
    Implements a basic pairs trading strategy based on the spread z-score.
    Exits a trade when the absolute z-score falls below z_exit.
    """
    etf, synth = etf.align(synth, join="inner")
    z = compute_spread_zscore(etf, synth, window)

    # Calculate volatility-based transaction costs if not provided as a fixed float
    ret_e = etf.pct_change()
    ret_s = synth.pct_change()
    spread_ret = ret_e - ret_s

    if transaction_cost is None:
        base_cost = 0.001  # 10 bps
        scaling_factor = 5
        tc_series = base_cost + scaling_factor * spread_ret.rolling(window=20).std()
        tc_series = tc_series.fillna(method='bfill').clip(lower=0)
    else:
        # Use fixed cost if provided as float
        tc_series = pd.Series(transaction_cost, index=etf.index)

    pos = pd.Series(0, index=z.index)
    in_pos = False
    for t in range(window, len(z)):
        if not in_pos:
            if z.iloc[t] > z_entry:
                pos.iloc[t] = -1
                in_pos = True
            elif z.iloc[t] < -z_entry:
                pos.iloc[t] = 1
                in_pos = True
        else:
            if abs(z.iloc[t]) < z_exit:
                pos.iloc[t] = 0
                in_pos = False
            else:
                pos.iloc[t] = pos.iloc[t-1]
    ret_e = etf.pct_change()
    ret_s = synth.pct_change()
    strat_ret = pos.shift(1) * (ret_e - ret_s)
    tc_adjustment = tc_series * pos.diff().abs().fillna(0)
    strat_ret = strat_ret - tc_adjustment
    strat_ret.fillna(0, inplace=True)
    cum_pnl = (1 + strat_ret).cumprod()
    ann_factor = np.sqrt(252)
    sharpe = ann_factor * strat_ret.mean() / strat_ret.std() if strat_ret.std() != 0 else np.nan
    vol = strat_ret.std() * ann_factor
    trades = (pos.diff().abs() > 0).sum()
    return strat_ret, cum_pnl, z, pos, sharpe, vol, trades

def performance_metrics(strat_returns, pos):
    """
    Computes performance metrics: Sharpe ratio, volatility, and number of trades.
    """
    ann_factor = np.sqrt(252)
    avg_ret = strat_returns.mean()
    vol = strat_returns.std() * ann_factor
    sharpe = (avg_ret / strat_returns.std()) * ann_factor if strat_returns.std() != 0 else np.nan
    trades = (pos.diff().abs() > 0).sum()
    return sharpe, vol, trades

# ---------------------------------------------------------------------
# 5. Main Execution
# ---------------------------------------------------------------------
def main(args):
    START_DATE = args.start
    END_DATE = args.end
    transaction_cost = args.tc
    logging.info(f"Running simulation from {START_DATE} to {END_DATE}")

    # Define tickers for ETFs and crypto tokens.
    etf_tickers = ["BITW", "GDLC"]
    token_tickers = ["BTC-USD", "ETH-USD", "LTC-USD", "XRP-USD", "BCH-USD"]

    etf_prices = download_data(etf_tickers, START_DATE, END_DATE, interval="1d")
    token_prices = download_data(token_tickers, START_DATE, END_DATE, interval="1d")
    if token_prices.empty:
        logging.error("Token data empty – check tickers/date range.")
        raise ValueError("Token data empty – check tickers/date range.")

    # Market cap weights for tokens.
    mcap_weights = {
        "BTC-USD": 0.5,
        "ETH-USD": 0.3,
        "LTC-USD": 0.1,
        "XRP-USD": 0.05,
        "BCH-USD": 0.05
    }

    # Choose a benchmark ETF.
    chosen_etf = None
    for t in etf_tickers:
        if t in etf_prices.columns and not etf_prices[t].empty:
            chosen_etf = etf_prices[t]
            break
    if chosen_etf is None:
        logging.warning("No suitable ETF found; defaulting initial value to 1.0.")
        initial_value = 1.0
    else:
        initial_value = chosen_etf.iloc[0]

    # Simulate synthetic portfolios.
    mcap_port = simulate_marketcap_portfolio(token_prices, mcap_weights, freq_str="1D", initial_value=initial_value)
    rp_port = simulate_risk_parity_portfolio(token_prices, window=20, freq_str="1D", initial_value=initial_value)

    # Compute tracking metrics for Market-Cap vs. ETF.
    te_m, _ = (compute_tracking_metrics(chosen_etf, mcap_port, window=20) if chosen_etf is not None else (None, None))
    zs_m = (compute_spread_zscore(chosen_etf, mcap_port, window=20) if chosen_etf is not None else None)

    # Compute tracking error for Risk-Parity vs. ETF.
    te_rp, _ = (compute_tracking_metrics(chosen_etf, rp_port, window=20) if chosen_etf is not None else (None, None))

    # ----------------
    # Cointegration Check
    # ----------------
    if chosen_etf is not None:
        score_mcap, pvalue_mcap = check_cointegration(chosen_etf, mcap_port)
        score_rp, pvalue_rp = check_cointegration(chosen_etf, rp_port)
        print("Cointegration test results:")
        print("ETF vs MarketCap: score = {:.4f}, p-value = {:.4f}".format(score_mcap, pvalue_mcap))
        print("ETF vs RiskParity: score = {:.4f}, p-value = {:.4f}".format(score_rp, pvalue_rp))

    def run_pairs(name, etf_ser, synth_ser, z_entry=1.0, z_exit=0.3, window=20):
        strat_ret, cum_pnl, z, pos, sharpe, vol, trades = pairs_backtest(
            etf_ser, synth_ser, z_entry, z_exit, window, transaction_cost=transaction_cost
        )
        return {
            "name": name,
            "strategy_rets": strat_ret,
            "cum_pnl": cum_pnl,
            "zscore": z,
            "pos": pos,
            "sharpe": sharpe,
            "vol": vol,
            "trades": trades
        }

    # --- Nested Grid Search for Optimal z_entry and z_exit Thresholds ---

    # --- Separate Grid Searches for Market‑Cap and Risk‑Parity Strategies ---
    z_entry_candidates = np.linspace(0.5, 2.0, 20)
    z_exit_candidates = np.linspace(0.1, 0.5, 20)

    # Storage for best results
    best_results = {
        'Market-Cap': {'sharpe': -np.inf, 'z_entry': None, 'z_exit': None},
        'Risk-Parity': {'sharpe': -np.inf, 'z_entry': None, 'z_exit': None}
    }

    # Grid search for Market‑Cap
    for z_entry in z_entry_candidates:
        for z_exit in z_exit_candidates:
            if z_exit >= z_entry:
                continue
            res_temp = run_pairs("Market-Cap", chosen_etf, mcap_port, z_entry=z_entry, z_exit=z_exit, window=20)
            if res_temp["sharpe"] > best_results['Market-Cap']['sharpe']:
                best_results['Market-Cap'] = {
                    'sharpe': res_temp["sharpe"],
                    'z_entry': z_entry,
                    'z_exit': z_exit
                }

    # Grid search for Risk‑Parity
    for z_entry in z_entry_candidates:
        for z_exit in z_exit_candidates:
            if z_exit >= z_entry:
                continue
            res_temp = run_pairs("Risk-Parity", chosen_etf, rp_port, z_entry=z_entry, z_exit=z_exit, window=20)
            if res_temp["sharpe"] > best_results['Risk-Parity']['sharpe']:
                best_results['Risk-Parity'] = {
                    'sharpe': res_temp["sharpe"],
                    'z_entry': z_entry,
                    'z_exit': z_exit
                }

    print("Optimal Market‑Cap thresholds:", best_results['Market-Cap'])
    print("Optimal Risk‑Parity thresholds:", best_results['Risk-Parity'])


# ---------------------------------------------------------------------
# 6. Plotting Results
# ---------------------------------------------------------------------

# -- New Plot: Comparison using Normalized Values --
    plt.figure(figsize=(12, 6))
    norm_etf = chosen_etf / chosen_etf.iloc[0]
    norm_mcap = mcap_port / mcap_port.iloc[0]
    norm_rp = rp_port / rp_port.iloc[0]
    norm_etf.plot(label="ETF Benchmark")
    norm_mcap.plot(label="Market Cap Strategy")
    norm_rp.plot(label="Risk Parity Strategy")
    plt.title("Comparison of ETF Benchmark, Market Cap, and Risk Parity (Normalized)")
    plt.xlabel("Date")
    plt.ylabel("Normalized Portfolio Value")
    plt.legend()
    plt.grid(True)
    plt.show()

    # -- New Plot: Raw Dollar Values --
    plt.figure(figsize=(12, 6))
    chosen_etf.plot(label="ETF Benchmark")
    mcap_port.plot(label="Market Cap Strategy")
    rp_port.plot(label="Risk Parity Strategy")
    plt.title("ETF vs. Market Cap vs. Risk Parity (Dollar Values)")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value (Dollars)")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot grid search results for Market‑Cap
    mcap_grid = []
    for z_entry in z_entry_candidates:
        for z_exit in z_exit_candidates:
            if z_exit >= z_entry:
                continue
            res_temp = run_pairs("Market-Cap", chosen_etf, mcap_port, z_entry=z_entry, z_exit=z_exit)
            mcap_grid.append((z_entry, z_exit, res_temp["sharpe"]))

    mcap_grid = np.array(mcap_grid)
    plt.figure(figsize=(10, 6))
    sc = plt.scatter(mcap_grid[:,0], mcap_grid[:,1], c=mcap_grid[:,2], cmap="Reds", s=100)
    plt.colorbar(sc, label="Sharpe Ratio")
    plt.xlabel("z_entry Threshold")
    plt.ylabel("z_exit Threshold")
    plt.title("Market‑Cap Grid Search: Sharpe Ratio")
    plt.grid(True)
    plt.show()

    # Plot grid search results for Risk-Parity
    rp_grid = []
    for z_entry in z_entry_candidates:
        for z_exit in z_exit_candidates:
            if z_exit >= z_entry:
                continue
            res_temp = run_pairs("Risk-Parity", chosen_etf, rp_port,
                                z_entry=z_entry, z_exit=z_exit)
            rp_grid.append((z_entry, z_exit, res_temp["sharpe"]))

    rp_grid = np.array(rp_grid)
    plt.figure(figsize=(10, 6))
    sc = plt.scatter(rp_grid[:,0], rp_grid[:,1], c=rp_grid[:,2], cmap="Blues", s=100)
    plt.colorbar(sc, label="Sharpe Ratio")
    plt.xlabel("z_entry Threshold")
    plt.ylabel("z_exit Threshold")
    plt.title("Risk-Parity Grid Search: Sharpe Ratio")
    plt.grid(True)
    plt.show()


    # Use the optimal thresholds for final pairs trading simulation.
    mcap_thresholds = best_results['Market-Cap']
    rp_thresholds   = best_results['Risk-Parity']

    res_mcap = run_pairs("Market-Cap", chosen_etf, mcap_port,
                        z_entry=mcap_thresholds['z_entry'],
                        z_exit=mcap_thresholds['z_exit'])

    res_rpar = run_pairs("Risk-Parity", chosen_etf, rp_port,
                        z_entry=rp_thresholds['z_entry'],
                        z_exit=rp_thresholds['z_exit'])

    # -- New Plot: Buy/Sell Signals on Spread Z‑Score (Risk‑Parity Strategy) --
    pos_rp = res_rpar["pos"]
    signal_changes_rp = pos_rp[pos_rp.diff() != 0]
    buy_signals_rp = signal_changes_rp[signal_changes_rp == 1]
    sell_signals_rp = signal_changes_rp[signal_changes_rp == -1]
    plt.figure(figsize=(12, 6))
    zscore_rp = res_rpar["zscore"]
    zscore_rp.plot(label="Spread Z‑Score", color="blue")
    rp_entry = rp_thresholds['z_entry']
    plt.axhline(0, color="black", linestyle="--")
    plt.axhline( rp_entry, color="red", linestyle="--")
    plt.axhline(-rp_entry, color="green", linestyle="--")
    plt.scatter(buy_signals_rp.index, zscore_rp.loc[buy_signals_rp.index], marker="^", color="g",
                s=100, label="Buy Signal")
    plt.scatter(sell_signals_rp.index, zscore_rp.loc[sell_signals_rp.index], marker="v", color="r",
                s=100, label="Sell Signal")
    plt.title("Spread Z‑Score with Buy/Sell Signals (Risk‑Parity Strategy)")
    plt.xlabel("Date")
    plt.ylabel("Z‑Score")
    plt.legend()
    plt.grid(True)
    plt.show()

    # -- New Plot: Buy/Sell Signals on Spread Z‑Score (Market‑Cap Strategy) --
    pos = res_mcap["pos"]
    signal_changes = pos[pos.diff() != 0]
    buy_signals = signal_changes[signal_changes == 1]
    sell_signals = signal_changes[signal_changes == -1]
    plt.figure(figsize=(12, 6))
    zscore = res_mcap["zscore"]
    zscore.plot(label="Spread Z‑Score", color="blue")
    plt.axhline(0, color="black", linestyle="--")
    mcap_entry = mcap_thresholds['z_entry']
    plt.axhline( mcap_entry, color="red", linestyle="--")
    plt.axhline(-mcap_entry, color="green", linestyle="--")
    plt.scatter(buy_signals.index, zscore.loc[buy_signals.index], marker="^", color="g",
                s=100, label="Buy Signal")
    plt.scatter(sell_signals.index, zscore.loc[sell_signals.index], marker="v", color="r",
                s=100, label="Sell Signal")
    plt.title("Spread Z‑Score with Buy/Sell Signals (Market‑Cap Strategy)")
    plt.xlabel("Date")
    plt.ylabel("Z‑Score")
    plt.legend()
    plt.grid(True)
    plt.show()

    # Plot synthetic portfolios.
    plt.figure(figsize=(10, 4))
    mcap_port.plot()
    plt.title("Market-Cap Synthetic Portfolio (Daily Rebalancing)")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value (Dollars)")
    plt.grid(True)
    plt.show()

    plt.figure(figsize=(10, 4))
    rp_port.plot()
    plt.title("Risk‑Parity Synthetic Portfolio (Daily Rebalancing)")
    plt.xlabel("Date")
    plt.ylabel("Portfolio Value (Dollars)")
    plt.grid(True)
    plt.show()

    if chosen_etf is not None:
        # -- New Plot: Combined Tracking Error for Risk‑Parity vs. ETF and Market‑Cap vs. ETF --
        plt.figure(figsize=(10, 4))
        if te_m is not None:
            te_m.plot(label="Market‑Cap Tracking Error", color="red")
        if te_rp is not None:
            te_rp.plot(label="Risk‑Parity Tracking Error", color="blue")
        plt.title("Tracking Error (20‑day) – Combined")
        plt.xlabel("Date")
        plt.ylabel("Tracking Error")
        plt.legend()
        plt.grid(True)
        plt.show()

        # Plot spread z-score for Market‑Cap vs. ETF.
        plt.figure(figsize=(10, 4))
        zs_m.plot()
        plt.axhline(1, linestyle="--", color="red")
        plt.axhline(-1, linestyle="--", color="green")
        plt.axhline(0, color="black")
        plt.title("Spread Z‑Score (ETF vs. Market‑Cap) – 20‑day Window")
        plt.xlabel("Date")
        plt.ylabel("Z‑Score")
        plt.grid(True)
        plt.show()

        # Plot pairs trading results.
        fig, ax = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
        for subplot_ax, res, title in zip(ax, [res_mcap, res_rpar], ["Market‑Cap leg", "Risk‑Parity leg"]):
            res["cum_pnl"].plot(ax=subplot_ax)
            subplot_ax.set_title(f"Pairs Trading – {title}")
            subplot_ax.set_xlabel("Date")
            subplot_ax.set_ylabel("Cumulative Return")
            txt = f"Sharpe: {res['sharpe']:.2f}\nVol: {res['vol']:.2%}\nTrades: {res['trades']}"
            subplot_ax.text(0.02, 0.95, txt, transform=subplot_ax.transAxes,
                            ha="left", va="top", fontsize=10,
                            bbox=dict(facecolor="white", alpha=0.7))
        plt.tight_layout()
        plt.show()

    else:
        logging.warning("No benchmark ETF data available; skipping ETF-related plots.")

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Run portfolio simulation and pairs trading backtest")
    parser.add_argument("--start", type=str, default="2022-01-01", help="Start date in YYYY-MM-DD")
    parser.add_argument("--end", type=str, default="2022-12-31", help="End date in YYYY-MM-DD")
    parser.add_argument("--tc", type=float, default=0.0, help="Transaction cost per trade (as a decimal)")
    args, unknown = parser.parse_known_args()
    main(args)

ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['XRP-USD', 'LTC-USD']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')
<ipython-input-49-7bcebd7a7ee9>:131: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  ret_s = synth.pct_change()


MissingDataError: exog contains inf or nans

### Using BTC as Benchmark (unnecessary)

In [ ]:
import argparse
import logging
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import coint

# ---------------------------------------------------------------------
# 0. Logging & Utils
# ---------------------------------------------------------------------
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

def download_data(tickers, start, end, interval="1d"):
    df = yf.download(tickers, start=start, end=end, interval=interval, progress=False)
    if df.empty:
        raise RuntimeError(f"No data for {tickers}")
    px = df.get("Adj Close", df.get("Close", df))
    return px.ffill()

# ---------------------------------------------------------------------
# 1. Rebalancing Simulators
# ---------------------------------------------------------------------
def get_rebalance_dates(prices, freq="1D"):
    raw = pd.date_range(prices.index[0], prices.index[-1], freq=freq)
    out = []
    for d in raw:
        i = prices.index.searchsorted(d)
        if i < len(prices):
            out.append(prices.index[i])
    return sorted(set(out))

def simulate_rebalanced(prices, weight_func, freq="1D", init=1.0):
    dates = get_rebalance_dates(prices, freq)
    if prices.index[0] not in dates:
        dates.insert(0, prices.index[0])
    dates = sorted(set(dates))
    port = pd.Series(index=prices.index, dtype=float)
    val = init

    for i, d in enumerate(dates):
        w = weight_func(prices, d)
        if np.isnan(w).any():
            w = np.ones(len(w)) / len(w)
        end = dates[i+1] if i+1 < len(dates) else prices.index[-1]
        window = prices.loc[d:end]
        rel = window.divide(window.loc[d], axis=1)
        pnl = rel.dot(w) * val
        port.loc[window.index] = pnl
        val = pnl.iloc[-1]
    return port

def simulate_marketcap(prices, mcap_weights, **kw):
    return simulate_rebalanced(
        prices,
        lambda px, d: np.array([mcap_weights.get(t, 0) for t in px.columns]),
        **kw
    )

def simulate_riskparity(prices, window=20, **kw):
    def wfunc(px, d):
        hist = px.loc[:d].iloc[-window:]
        if hist.shape[0] < 2:
            return np.ones(px.shape[1]) / px.shape[1]
        vol = np.log(hist / hist.shift(1)).dropna().std().replace(0, 1e-8)
        inv = 1 / vol
        return (inv / inv.sum()).values
    return simulate_rebalanced(prices, wfunc, **kw)

# ---------------------------------------------------------------------
# 2. Pairs Backtest (fixed z‐score)
# ---------------------------------------------------------------------
def pairs_backtest(bm, synth, z_entry, z_exit, window=20, tc=None):
    bm, synth = bm.align(synth, join="inner")
    spread = bm - synth
    mu = spread.rolling(window).mean()
    sigma = spread.rolling(window).std()
    z = (spread - mu) / sigma

    re, rs = bm.pct_change(), synth.pct_change()
    spread_ret = re - rs

    if tc is None:
        tc_series = 0.001 + 5 * spread_ret.rolling(window).std()
        tc_series = tc_series.fillna(method="bfill").clip(lower=0)
    else:
        tc_series = pd.Series(tc, index=bm.index)

    pos = pd.Series(0, index=z.index)
    in_pos = False
    for t in range(window, len(z)):
        if not in_pos:
            if z.iloc[t] >  z_entry:
                pos.iloc[t], in_pos = -1, True
            elif z.iloc[t] < -z_entry:
                pos.iloc[t], in_pos =  1, True
        else:
            if abs(z.iloc[t]) < z_exit:
                pos.iloc[t], in_pos = 0, False
            else:
                pos.iloc[t] = pos.iloc[t-1]

    strat = pos.shift(1) * spread_ret
    strat -= tc_series * pos.diff().abs().fillna(0)
    strat = strat.fillna(0)
    cum = (1 + strat).cumprod()
    ann = np.sqrt(252)
    sr = ann * strat.mean() / strat.std() if strat.std() != 0 else np.nan
    vol = strat.std() * ann
    trades = int((pos.diff().abs() > 0).sum())

    return strat, cum, z, pos, sr, vol, trades

# ---------------------------------------------------------------------
# 3. Main
# ---------------------------------------------------------------------
def main(args):
    # -- download universe + BTC benchmark
    tokens = ["BTC-USD", "ETH-USD", "LTC-USD", "XRP-USD", "BCH-USD"]
    prices = download_data(tokens, args.start, args.end)
    btc    = prices["BTC-USD"]

    # -- simulate synthetics (init value = first btc price)
    init   = btc.iloc[0]
    mcap_w = {"BTC-USD":0.5,"ETH-USD":0.3,"LTC-USD":0.1,"XRP-USD":0.05,"BCH-USD":0.05}
    mcap_port = simulate_marketcap(prices, mcap_w, freq="1D", init=init)
    rp_port   = simulate_riskparity(prices, window=20, freq="1D", init=init)

    # -- test cointegration
    stat_m, p_m, _ = coint(btc, mcap_port)
    stat_r, p_r, _ = coint(btc, rp_port)
    print(f"Cointegration BTC vs Market-Cap: stat={stat_m:.3f}, p={p_m:.3f}")
    print(f"Cointegration BTC vs Risk-Parity: stat={stat_r:.3f}, p={p_r:.3f}")

    # -----------------------------------------------------------------
    # 3.x Grid-Search thresholds for BTC benchmark
    # -----------------------------------------------------------------
    z_entry_candidates = np.linspace(0.5, 2.0, 20)
    z_exit_candidates  = np.linspace(0.1, 0.5, 20)

    best_mc = {'sharpe': -np.inf, 'z_entry': None, 'z_exit': None}
    best_rp = {'sharpe': -np.inf, 'z_entry': None, 'z_exit': None}

    for ze in z_entry_candidates:
        for zx in z_exit_candidates:
            if zx >= ze:
                continue

            _, _, _, _, sr_mc, _, _ = pairs_backtest(btc, mcap_port, ze, zx, 20, args.tc)
            if sr_mc > best_mc['sharpe']:
                best_mc.update({'sharpe': sr_mc, 'z_entry': ze, 'z_exit': zx})

            _, _, _, _, sr_rp, _, _ = pairs_backtest(btc, rp_port, ze, zx, 20, args.tc)
            if sr_rp > best_rp['sharpe']:
                best_rp.update({'sharpe': sr_rp, 'z_entry': ze, 'z_exit': zx})

    print("Best BTC vs Market-Cap:", best_mc)
    print("Best BTC vs Risk-Parity:", best_rp)

    # -- final backtests using BTC-optimized thresholds
    ze_mc, zx_mc = best_mc['z_entry'], best_mc['z_exit']
    ze_rp, zx_rp = best_rp['z_entry'], best_rp['z_exit']

    _, cum_mc, *_ , sr_mc, vol_mc, tr_mc = pairs_backtest(btc, mcap_port, ze_mc, zx_mc, 20, args.tc)
    _, cum_rp, *_ , sr_rp, vol_rp, tr_rp = pairs_backtest(btc, rp_port,   ze_rp, zx_rp, 20, args.tc)

    # -- twin plots with aligned boxes & grids
    fig, axes = plt.subplots(1, 2, figsize=(14, 4), sharey=True)
    for ax, cum, sr, vol, tr, lbl in zip(
        axes,
        [cum_mc, cum_rp],
        [sr_mc, sr_rp],
        [vol_mc, vol_rp],
        [tr_mc, tr_rp],
        ["Market-Cap", "Risk-Parity"]
    ):
        ax.plot(cum, label=lbl)
        ax.set_title(f"{lbl} leg")
        ax.set_xlabel("Date")
        ax.set_ylabel("Cumulative Return")
        txt = f"Sharpe: {sr:.2f}\nVol: {vol:.2%}\nTrades: {tr}"
        ax.text(0.02, 0.95, txt, transform=ax.transAxes,
                ha="left", va="top", fontsize=10,
                bbox=dict(facecolor="white", alpha=0.7))
        ax.grid(True)

    plt.tight_layout()
    plt.show()

if __name__ == "__main__":
    p = argparse.ArgumentParser()
    p.add_argument("--start", type=str, default="2022-01-01")
    p.add_argument("--end",   type=str, default="2022-12-31")
    p.add_argument("--tc",    type=float, default=0.0)
    args, _ = p.parse_known_args()
    main(args)


ERROR:yfinance:
2 Failed downloads:
ERROR:yfinance:['XRP-USD', 'LTC-USD']: YFRateLimitError('Too Many Requests. Rate limited. Try after a while.')


KeyError: 'BTC-USD'

SyntaxError: unterminated string literal (detected at line 19) (<ipython-input-27-3b121c6ebe91>, line 19)